# Step 1: Extract Data from Azure Synapse to ADLS

This notebook extracts data from Azure Synapse Dedicated SQL Pool to Azure Data Lake Storage Gen2 (ADLS).

## Process Overview
1. Configure connection parameters
2. Connect to source database
3. Setup external objects (credential, data source, file format)
4. Discover tables to migrate
5. Extract tables to ADLS in Parquet format
6. Validate extraction results

## Prerequisites
- Access to Azure Synapse Dedicated SQL Pool
- Access to ADLS Gen2 storage account
- Appropriate permissions (see PERMISSIONS_GUIDE.md)
- ODBC Driver 17 for SQL Server installed

## Configuration

Update the configuration parameters below with your environment details.

In [ ]:
# Source Azure Synapse Configuration
source_server = "<your-synapse-server>.sql.azuresynapse.net"
source_database = "<your-database-name>"

# Azure Data Lake Storage Configuration
storage_account = "<your-storage-account>"
container = "migration-staging"

# Migration Settings
enable_partitioning = True  # Enable partitioning for large tables (> 1 GB)
batch_size = 50  # Number of tables to process in each batch

# Authentication Configuration
# Options: 'token', 'interactive', 'sql'
auth_type = 'interactive'  # Use 'token' in Fabric notebooks with managed identity

print("Configuration loaded successfully ✓")
print(f"Source: {source_server}/{source_database}")
print(f"Storage: {storage_account}/{container}")

## Setup and Import Helper Functions

Load the migration helper functions for database connections and utilities.

In [ ]:
# Import helper functions
import sys
sys.path.append('/lakehouse/default/Files/notebooks/utils')

from migration_helpers import ConnectionHelper, MigrationUtils, Colors
import time
from datetime import datetime

print("Helper functions imported successfully ✓")

## Connect to Source Database

Establish connection to Azure Synapse Dedicated SQL Pool.

In [ ]:
# Get authentication token if using Fabric notebook
auth_config = {}

if auth_type == 'token':
    # Get token from Fabric runtime
    token = ConnectionHelper.get_spark_token("https://database.windows.net/.default")
    auth_config = {'auth_type': 'token', 'token': token}
else:
    auth_config = {'auth_type': auth_type}

# Connect to source database
source_conn = ConnectionHelper.connect_azure_sql(source_server, source_database, auth_config)

print("\n" + "="*70)
print("Connection established successfully!")
print("="*70)

## Setup External Objects

Create external objects required for data extraction:
- Database scoped credential
- External data source pointing to ADLS
- External file format (Parquet)

In [ ]:
# Setup external objects
setup_success = MigrationUtils.setup_external_objects(source_conn, storage_account, container)

if not setup_success:
    raise Exception("Failed to setup external objects. Please check permissions and retry.")

print("\n" + "="*70)
print("External objects created successfully!")
print("="*70)

## Discover Tables to Extract

Scan the source database to identify all tables that need to be migrated.

In [ ]:
# Get list of tables
tables = MigrationUtils.get_tables_list(source_conn)

if len(tables) == 0:
    print(f"{Colors.YELLOW}⚠️  No tables found to extract{Colors.END}")
else:
    print(f"\n{Colors.GREEN}✅ Ready to extract {len(tables)} tables{Colors.END}")
    
    # Store tables for reference
    table_list = [(schema, table, rows, size) for schema, table, rows, size in tables]
    
    print(f"\nTotal data volume: {sum(t[3] for t in tables):.2f} GB")
    print(f"Total rows: {sum(t[2] for t in tables):,}")

## Extract Tables to ADLS

Extract each table to ADLS Gen2 using CREATE EXTERNAL TABLE AS SELECT (CETAS).

This process:
- Creates external tables pointing to ADLS
- Exports data in Parquet format with Snappy compression
- Organizes data by schema/table folder structure
- Tracks progress and errors

In [ ]:
# Initialize tracking
extraction_stats = {
    'total': len(tables),
    'extracted': 0,
    'failed': 0,
    'start_time': datetime.now()
}

failed_tables = []

print("\n" + "="*70)
print("Starting Table Extraction")
print("="*70 + "\n")

cursor = source_conn.cursor()

for idx, (schema, table, row_count, size_gb) in enumerate(tables, 1):
    try:
        print(f"\n[{idx}/{len(tables)}] Extracting [{schema}].[{table}]...")
        print(f"   Rows: {row_count:,} | Size: {size_gb:.2f} GB")
        
        start_time = time.time()
        
        # Drop external table if exists
        external_table_name = f"ext_{table}_migration"
        cursor.execute(f"""
            IF EXISTS (SELECT * FROM sys.external_tables WHERE name = '{external_table_name}')
                DROP EXTERNAL TABLE [{schema}].[{external_table_name}]
        """)
        
        # Create external table with CETAS
        location = f"{schema}/{table}/"
        
        cursor.execute(f"""
            CREATE EXTERNAL TABLE [{schema}].[{external_table_name}]
            WITH (
                LOCATION = '{location}',
                DATA_SOURCE = MigrationStaging,
                FILE_FORMAT = ParquetFormat
            )
            AS
            SELECT * FROM [{schema}].[{table}]
        """)
        
        source_conn.commit()
        
        duration = time.time() - start_time
        extraction_stats['extracted'] += 1
        
        print(f"{Colors.GREEN}   ✅ Completed in {duration:.1f}s{Colors.END}")
        print(f"   Progress: {extraction_stats['extracted']}/{extraction_stats['total']}")
        
    except Exception as e:
        extraction_stats['failed'] += 1
        failed_tables.append((schema, table, str(e)))
        print(f"{Colors.RED}   ❌ Failed: {e}{Colors.END}")
        continue

extraction_stats['end_time'] = datetime.now()

# Print summary
duration = (extraction_stats['end_time'] - extraction_stats['start_time']).total_seconds()

print("\n" + "="*70)
print("EXTRACTION SUMMARY")
print("="*70)
print(f"Total tables:     {extraction_stats['total']}")
print(f"Extracted:        {Colors.GREEN}{extraction_stats['extracted']}{Colors.END}")
print(f"Failed:           {Colors.RED}{extraction_stats['failed']}{Colors.END}")
print(f"Duration:         {duration:.1f} seconds ({duration/60:.1f} minutes)")
print(f"Start time:       {extraction_stats['start_time'].strftime('%Y-%m-%d %H:%M:%S')}")
print(f"End time:         {extraction_stats['end_time'].strftime('%Y-%m-%d %H:%M:%S')}")
print("="*70)

if extraction_stats['failed'] > 0:
    print(f"\n{Colors.YELLOW}⚠️  Failed Tables:{Colors.END}")
    for schema, table, error in failed_tables:
        print(f"  - [{schema}].[{table}]: {error}")
else:
    print(f"\n{Colors.GREEN}✅ All tables extracted successfully!{Colors.END}")

## Cleanup and Close Connection

Clean up external tables and close database connection.

In [ ]:
# Close connection
if source_conn:
    source_conn.close()
    print(f"{Colors.GREEN}✅ Connection closed{Colors.END}")

print("\n" + "="*70)
print("Extraction process completed!")
print("="*70)
print("\nNext steps:")
print("1. Verify data in ADLS: Check the container for extracted Parquet files")
print("2. Run notebook '02_load_data.ipynb' to load data into Fabric Warehouse")

## Verify Extraction (Optional)

You can verify the extracted data by checking the ADLS container or using Azure Storage Explorer.

Expected folder structure:
```
migration-staging/
├── schema1/
│   ├── table1/
│   │   └── *.parquet
│   └── table2/
│       └── *.parquet
└── schema2/
    └── table3/
        └── *.parquet
```